In [ ]:
class DataQualityEvaluator:
    def __init__(self, dataframe):
        self.df = dataframe

    def completeness(self):
        missing_count = self.df.isnull().sum().sum()
        total_count = self.df.size
        completeness_score = (total_count - missing_count) / total_count
        return completeness_score

    def range_validity(self, column, min_val, max_val):
        if self.df[column].dtype in ['int64', 'float64']:
            valid_count = self.df[column].apply(lambda x: min_val <= x <= max_val).sum()
            total_count = self.df[column].count()
            return valid_count / total_count
        else:
            return None

    def check_outliers(self, column):
        if self.df[column].dtype in ['int64', 'float64']:
            mean = self.df[column].mean()
            std = self.df[column].std()
            outliers_count = self.df[column].apply(lambda x: abs(x - mean) > 3 * std).sum()
            total_count = self.df[column].count()
            return (total_count - outliers_count) / total_count
        else:
            return None

    def uniqueness(self):
        unique_scores = [self.df[col].nunique() / len(self.df[col]) if self.df[col].dtype in ['int64', 'float64', 'object'] else 1 for col in self.df.columns]
        return sum(unique_scores) / len(unique_scores)

    def calculate_overall_quality(self):
        completeness_score = self.completeness()
        range_validity_scores = [self.range_validity(col, 0, 100) for col in self.df.columns if self.df[col].dtype in ['int64', 'float64']]
        outlier_scores = [self.check_outliers(col) for col in self.df.columns if self.df[col].dtype in ['int64', 'float64']]
        uniqueness_score = self.uniqueness()

        range_validity_scores = [score for score in range_validity_scores if score is not None]
        outlier_scores = [score for score in outlier_scores if score is not None]

        range_validity_avg = sum(range_validity_scores) / len(range_validity_scores) if range_validity_scores else 1
        outlier_avg = sum(outlier_scores) / len(outlier_scores) if outlier_scores else 1

        overall_score = (completeness_score + range_validity_avg + outlier_avg + uniqueness_score) / 4
        return overall_score

    def quality_color_indicator(self):
        quality_score = self.calculate_overall_quality() * 100
        color = "Red"
        if quality_score >= 70:
            color = "Green"
        elif 50 <= quality_score < 70:
            color = "Orange"

        return f"Your result is: {quality_score:.2f}% {color}"


In [ ]:
import pandas as pd

df = pd.read_csv('animes.csv')
df2=df.dropna()

In [ ]:
evaluator = DataQualityEvaluator(df)

print(evaluator.quality_color_indicator())